<a href="https://colab.research.google.com/github/jsansao/transformers_pt/blob/main/BERTimbau_Fine_tuning_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inferência usando transformers pré-treinados do HuggingFace 

In [1]:
#@title Passo 1:Instalando Hugging Face Transformers
# We won't need TensorFlow here
!pip uninstall -y tensorflow
# Install `transformers` from master
!pip install transformers datasets
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.9.1
# tokenizers version at notebook update --- 0.7.0

Found existing installation: tensorflow 2.8.2+zzzcolab20220929150707
Uninstalling tensorflow-2.8.2+zzzcolab20220929150707:
  Successfully uninstalled tensorflow-2.8.2+zzzcolab20220929150707
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 26.6 MB/s 
     |████████████████████████████████| 432 kB 63.4 MB/s 
     |████████████████████████████████| 163 kB 70.5 MB/s 
     |████████████████████████████████| 7.6 MB 54.7 MB/s 
     |████████████████████████████████| 115 kB 70.9 MB/s 
     |████████████████████████████████| 212 kB 71.1 MB/s 
     |████████████████████████████████| 127 kB 68.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the fol

In [2]:
#@title Step 1: Loading the Dataset
#1.Load kant.txt using the Colab file manager
#2.Downloading the file from GitHub
!curl -L https://raw.githubusercontent.com/jsansao/corpus_ptbr/main/dump_Machado_Nobreak.txt --output "dump.txt"
#!curl -L https://raw.githubusercontent.com/jsansao/corpus_ptbr/main/Lyrics_ChicoBuarque.txt --output "kant.txt"
!awk NF < dump.txt > kant.txt


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1739k  100 1739k    0     0  1277k      0  0:00:01  0:00:01 --:--:-- 1277k


In [3]:
#@title Passo 2:Baixando e salvando BR_BERTo
#https://huggingface.co/rdenadai/BR_BERTo

from transformers import AutoTokenizer, AutoModelForMaskedLM

#tokenizer = AutoTokenizer.from_pretrained("rdenadai/BR_BERTo")
#model = AutoModelForMaskedLM.from_pretrained("rdenadai/BR_BERTo")

tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
model = AutoModelForMaskedLM.from_pretrained('neuralmind/bert-base-portuguese-cased')

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/210k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
from datasets import load_dataset
#datasets = load_dataset("text", data_files={"train": "kant.txt", "validation": "kant_teste.txt"})
#datasets = load_dataset("text", data_files={"train": "kant.txt"}, split='train[:90%]')
ds = load_dataset("text", data_files={"train": "kant.txt"})

datasets = ds["train"].train_test_split()

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-612e1dee769be6e0/0.0.0/21a506d1b2b34316b1e82d0bd79066905d846e5d7e619823c0dd338d6f1fa6ad. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [6]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

exemplo de entrada

In [7]:
tokenized_datasets["train"][4]

{'input_ids': [101,
  9225,
  222,
  3265,
  11962,
  202,
  16286,
  117,
  230,
  305,
  3976,
  1156,
  122,
  1256,
  3724,
  125,
  7406,
  2248,
  119,
  385,
  14881,
  22288,
  260,
  3724,
  117,
  17898,
  15292,
  22288,
  118,
  260,
  122,
  15231,
  18417,
  118,
  260,
  202,
  3147,
  117,
  240,
  5530,
  180,
  9461,
  119,
  107,
  1533,
  253,
  123,
  1407,
  305,
  3976,
  1156,
  107,
  117,
  10355,
  368,
  5747,
  576,
  119,
  761,
  146,
  596,
  117,
  7623,
  5288,
  5852,
  229,
  1288,
  117,
  5288,
  5726,
  117,
  320,
  1528,
  132,
  346,
  18420,
  2836,
  117,
  122,
  9679,
  5012,
  159,
  13486,
  2146,
  119,
  1683,
  6365,
  117,
  495,
  3695,
  117,
  346,
  641,
  22283,
  20576,
  117,
  449,
  2798,
  2745,
  253,
  20576,
  3102,
  1147,
  119,
  192,
  346,
  2036,
  10262,
  10347,
  22281,
  8410,
  695,
  17375,
  22282,
  324,
  132,
  260,
  10089,
  562,
  179,
  19636,
  16819,
  1075,
  171,
  10712,
  179,
  180,
  3512,
  10

In [8]:
#block_size = tokenizer.model_max_length
block_size = 32


In [9]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [10]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [11]:
#@title Step 11: Defining a Data Collator
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [12]:
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {metric.compute(predictions=predictions, references=labels)}

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
#@title Step 12: Initializing the Trainer
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./KantaiBERT",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=16,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["test"],    
)

In [14]:
#@title Step 13: Pre-training the Model
%%time
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 11065
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 6920
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,2.673500
1000,2.462700
1500,2.351400
2000,2.249200
2500,2.156100
3000,2.121100
3500,2.040300
4000,2.022000
4500,1.961700
5000,1.917000




Training completed. Do not forget to share your model on huggingface.co/models =)




CPU times: user 15min 54s, sys: 1min 15s, total: 17min 9s
Wall time: 17min 12s


TrainOutput(global_step=6920, training_loss=2.09629436338568, metrics={'train_runtime': 1032.6786, 'train_samples_per_second': 107.149, 'train_steps_per_second': 6.701, 'total_flos': 1820210317305600.0, 'train_loss': 2.09629436338568, 'epoch': 10.0})

In [15]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 3753
  Batch size = 8


Perplexity: 8.28


In [16]:
eval_results

{'eval_loss': 2.114231586456299,
 'eval_runtime': 9.3674,
 'eval_samples_per_second': 400.646,
 'eval_steps_per_second': 50.174,
 'epoch': 10.0}

In [17]:
#@title Step 14: Saving the Final Model(+tokenizer + config) to disk
trainer.save_model("./KantaiBERT")

Saving model checkpoint to ./KantaiBERT
Configuration saved in ./KantaiBERT/config.json
Model weights saved in ./KantaiBERT/pytorch_model.bin


In [18]:
#@title Step 3: Configurando o pipeline fill-mask
#@title Step 15: Language Modeling with the FillMaskPipeline
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./KantaiBERT",
    tokenizer=tokenizer
)

loading configuration file ./KantaiBERT/config.json
Model config BertConfig {
  "_name_or_path": "./KantaiBERT",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.23.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 29794
}

loading configuration file ./KantaiBERT/config.json
Model config BertConfig {
  "_nam

In [19]:
fill_mask("O rapaz olhou para o [MASK] ")

[{'score': 0.2352643460035324,
  'token': 119,
  'token_str': '.',
  'sequence': 'O rapaz olhou para o.'},
 {'score': 0.12906503677368164,
  'token': 8105,
  'token_str': 'chão',
  'sequence': 'O rapaz olhou para o chão'},
 {'score': 0.09621220082044601,
  'token': 1341,
  'token_str': 'lado',
  'sequence': 'O rapaz olhou para o lado'},
 {'score': 0.05336952209472656,
  'token': 8242,
  'token_str': 'céu',
  'sequence': 'O rapaz olhou para o céu'},
 {'score': 0.03085210919380188,
  'token': 2979,
  'token_str': 'alto',
  'sequence': 'O rapaz olhou para o alto'}]

In [20]:
fill_mask("A moça olhou para o [MASK] ")

[{'score': 0.16936281323432922,
  'token': 8105,
  'token_str': 'chão',
  'sequence': 'A moça olhou para o chão'},
 {'score': 0.1374182552099228,
  'token': 119,
  'token_str': '.',
  'sequence': 'A moça olhou para o.'},
 {'score': 0.12273475527763367,
  'token': 1341,
  'token_str': 'lado',
  'sequence': 'A moça olhou para o lado'},
 {'score': 0.06763889640569687,
  'token': 8242,
  'token_str': 'céu',
  'sequence': 'A moça olhou para o céu'},
 {'score': 0.03761054202914238,
  'token': 14148,
  'token_str': 'corredor',
  'sequence': 'A moça olhou para o corredor'}]

In [21]:
fill_mask("Comprou uma [MASK] na loja. ")

[{'score': 0.15394648909568787,
  'token': 3743,
  'token_str': 'carta',
  'sequence': 'Comprou uma carta na loja.'},
 {'score': 0.12847693264484406,
  'token': 4428,
  'token_str': 'nota',
  'sequence': 'Comprou uma nota na loja.'},
 {'score': 0.05174675211310387,
  'token': 9104,
  'token_str': 'cadeira',
  'sequence': 'Comprou uma cadeira na loja.'},
 {'score': 0.051646918058395386,
  'token': 576,
  'token_str': 'vez',
  'sequence': 'Comprou uma vez na loja.'},
 {'score': 0.033814750611782074,
  'token': 4320,
  'token_str': 'peça',
  'sequence': 'Comprou uma peça na loja.'}]

In [22]:
fill_mask("A mulher não é [MASK]. ")

[{'score': 0.08675719052553177,
  'token': 3874,
  'token_str': 'nada',
  'sequence': 'A mulher não é nada.'},
 {'score': 0.06793718785047531,
  'token': 1766,
  'token_str': 'má',
  'sequence': 'A mulher não é má.'},
 {'score': 0.05980519950389862,
  'token': 8540,
  'token_str': 'feliz',
  'sequence': 'A mulher não é feliz.'},
 {'score': 0.056523311883211136,
  'token': 19543,
  'token_str': 'bonita',
  'sequence': 'A mulher não é bonita.'},
 {'score': 0.0550476498901844,
  'token': 19462,
  'token_str': 'santa',
  'sequence': 'A mulher não é santa.'}]

In [23]:
fill_mask("O homem não é [MASK]. ")

[{'score': 0.14281556010246277,
  'token': 2397,
  'token_str': 'homem',
  'sequence': 'O homem não é homem.'},
 {'score': 0.11273413896560669,
  'token': 3874,
  'token_str': 'nada',
  'sequence': 'O homem não é nada.'},
 {'score': 0.04690055549144745,
  'token': 2538,
  'token_str': 'Deus',
  'sequence': 'O homem não é Deus.'},
 {'score': 0.04369949549436569,
  'token': 13380,
  'token_str': 'perfeito',
  'sequence': 'O homem não é perfeito.'},
 {'score': 0.036377403885126114,
  'token': 2124,
  'token_str': 'forte',
  'sequence': 'O homem não é forte.'}]

In [24]:
fill_mask("Ele é um bom [MASK]. ")

[{'score': 0.2664191424846649,
  'token': 2397,
  'token_str': 'homem',
  'sequence': 'Ele é um bom homem.'},
 {'score': 0.13522198796272278,
  'token': 13254,
  'token_str': 'rapaz',
  'sequence': 'Ele é um bom rapaz.'},
 {'score': 0.11699407547712326,
  'token': 3695,
  'token_str': 'amigo',
  'sequence': 'Ele é um bom amigo.'},
 {'score': 0.04700295254588127,
  'token': 6669,
  'token_str': 'advogado',
  'sequence': 'Ele é um bom advogado.'},
 {'score': 0.02809665910899639,
  'token': 4170,
  'token_str': 'marido',
  'sequence': 'Ele é um bom marido.'}]

In [25]:
fill_mask("Ela é uma boa [MASK]. ")

[{'score': 0.248652845621109,
  'token': 18073,
  'token_str': 'moça',
  'sequence': 'Ela é uma boa moça.'},
 {'score': 0.215536966919899,
  'token': 2760,
  'token_str': 'pessoa',
  'sequence': 'Ela é uma boa pessoa.'},
 {'score': 0.06778266280889511,
  'token': 8932,
  'token_str': 'amiga',
  'sequence': 'Ela é uma boa amiga.'},
 {'score': 0.0507914200425148,
  'token': 17704,
  'token_str': 'senhora',
  'sequence': 'Ela é uma boa senhora.'},
 {'score': 0.048672232776880264,
  'token': 2196,
  'token_str': 'mãe',
  'sequence': 'Ela é uma boa mãe.'}]

In [26]:
fill_mask("Faz de conta que ainda é [MASK]. ")

[{'score': 0.4922662079334259,
  'token': 1373,
  'token_str': 'tarde',
  'sequence': 'Faz de conta que ainda é tarde.'},
 {'score': 0.14644524455070496,
  'token': 596,
  'token_str': 'tempo',
  'sequence': 'Faz de conta que ainda é tempo.'},
 {'score': 0.10269620269536972,
  'token': 8545,
  'token_str': 'cedo',
  'sequence': 'Faz de conta que ainda é cedo.'},
 {'score': 0.03623180836439133,
  'token': 1695,
  'token_str': 'pouco',
  'sequence': 'Faz de conta que ainda é pouco.'},
 {'score': 0.02410014346241951,
  'token': 2397,
  'token_str': 'homem',
  'sequence': 'Faz de conta que ainda é homem.'}]

In [27]:
fill_mask("Depois da tempestade vem a [MASK]. ")

[{'score': 0.12820130586624146,
  'token': 1386,
  'token_str': 'morte',
  'sequence': 'Depois da tempestade vem a morte.'},
 {'score': 0.1122041568160057,
  'token': 13943,
  'token_str': 'lua',
  'sequence': 'Depois da tempestade vem a lua.'},
 {'score': 0.09491245448589325,
  'token': 9856,
  'token_str': 'chuva',
  'sequence': 'Depois da tempestade vem a chuva.'},
 {'score': 0.08212915062904358,
  'token': 11062,
  'token_str': 'fome',
  'sequence': 'Depois da tempestade vem a fome.'},
 {'score': 0.07224278897047043,
  'token': 7658,
  'token_str': 'tempestade',
  'sequence': 'Depois da tempestade vem a tempestade.'}]

In [28]:
fill_mask("Mais vale um pássaro na mão do que [MASK] voando. ")

[{'score': 0.20211467146873474,
  'token': 222,
  'token_str': 'um',
  'sequence': 'Mais vale um pássaro na mão do que um voando.'},
 {'score': 0.17814117670059204,
  'token': 5197,
  'token_str': 'sair',
  'sequence': 'Mais vale um pássaro na mão do que sair voando.'},
 {'score': 0.1234157383441925,
  'token': 14934,
  'token_str': 'voar',
  'sequence': 'Mais vale um pássaro na mão do que voar voando.'},
 {'score': 0.08933647722005844,
  'token': 1342,
  'token_str': 'outro',
  'sequence': 'Mais vale um pássaro na mão do que outro voando.'},
 {'score': 0.06783545762300491,
  'token': 4412,
  'token_str': 'ficar',
  'sequence': 'Mais vale um pássaro na mão do que ficar voando.'}]

In [29]:
fill_mask("Não tinha [MASK], mas almoçou mesmo assim. ")

[{'score': 0.28531768918037415,
  'token': 11062,
  'token_str': 'fome',
  'sequence': 'Não tinha fome, mas almoçou mesmo assim.'},
 {'score': 0.11301910877227783,
  'token': 11334,
  'token_str': 'sono',
  'sequence': 'Não tinha sono, mas almoçou mesmo assim.'},
 {'score': 0.10451152920722961,
  'token': 3874,
  'token_str': 'nada',
  'sequence': 'Não tinha nada, mas almoçou mesmo assim.'},
 {'score': 0.031166598200798035,
  'token': 7568,
  'token_str': 'café',
  'sequence': 'Não tinha café, mas almoçou mesmo assim.'},
 {'score': 0.02397201769053936,
  'token': 14890,
  'token_str': 'jantar',
  'sequence': 'Não tinha jantar, mas almoçou mesmo assim.'}]

In [30]:
fill_mask("Bebeu um copo d'água, pois tinha [MASK]. ")

[{'score': 0.26561281085014343,
  'token': 14594,
  'token_str': 'febre',
  'sequence': "Bebeu um copo d'água, pois tinha febre."},
 {'score': 0.19732101261615753,
  'token': 2496,
  'token_str': 'sede',
  'sequence': "Bebeu um copo d'água, pois tinha sede."},
 {'score': 0.13055835664272308,
  'token': 11062,
  'token_str': 'fome',
  'sequence': "Bebeu um copo d'água, pois tinha fome."},
 {'score': 0.051105644553899765,
  'token': 7672,
  'token_str': 'medo',
  'sequence': "Bebeu um copo d'água, pois tinha medo."},
 {'score': 0.04828227683901787,
  'token': 11334,
  'token_str': 'sono',
  'sequence': "Bebeu um copo d'água, pois tinha sono."}]

In [31]:
fill_mask("Sem saber, ele descobriu um [MASK]. ")

[{'score': 0.2938809096813202,
  'token': 11021,
  'token_str': 'segredo',
  'sequence': 'Sem saber, ele descobriu um segredo.'},
 {'score': 0.1432889699935913,
  'token': 16269,
  'token_str': 'mistério',
  'sequence': 'Sem saber, ele descobriu um mistério.'},
 {'score': 0.029571831226348877,
  'token': 3420,
  'token_str': 'caminho',
  'sequence': 'Sem saber, ele descobriu um caminho.'},
 {'score': 0.029519477859139442,
  'token': 3204,
  'token_str': 'plano',
  'sequence': 'Sem saber, ele descobriu um plano.'},
 {'score': 0.02110275626182556,
  'token': 1722,
  'token_str': 'livro',
  'sequence': 'Sem saber, ele descobriu um livro.'}]

In [32]:
fill_mask("Pedro fez fortuna vendendo [MASK]. ")

[{'score': 0.24614425003528595,
  'token': 5976,
  'token_str': 'escravos',
  'sequence': 'Pedro fez fortuna vendendo escravos.'},
 {'score': 0.1205301433801651,
  'token': 6691,
  'token_str': 'papéis',
  'sequence': 'Pedro fez fortuna vendendo papéis.'},
 {'score': 0.10114994645118713,
  'token': 2978,
  'token_str': 'livros',
  'sequence': 'Pedro fez fortuna vendendo livros.'},
 {'score': 0.06380639970302582,
  'token': 7568,
  'token_str': 'café',
  'sequence': 'Pedro fez fortuna vendendo café.'},
 {'score': 0.03742444887757301,
  'token': 20460,
  'token_str': 'doces',
  'sequence': 'Pedro fez fortuna vendendo doces.'}]

In [33]:
fill_mask("Sem medo de ser [MASK]. ")

[{'score': 0.7978298664093018,
  'token': 8540,
  'token_str': 'feliz',
  'sequence': 'Sem medo de ser feliz.'},
 {'score': 0.05636584386229515,
  'token': 7148,
  'token_str': 'padre',
  'sequence': 'Sem medo de ser padre.'},
 {'score': 0.009021664969623089,
  'token': 5139,
  'token_str': 'preso',
  'sequence': 'Sem medo de ser preso.'},
 {'score': 0.006950309034436941,
  'token': 3382,
  'token_str': 'visto',
  'sequence': 'Sem medo de ser visto.'},
 {'score': 0.0068481555208563805,
  'token': 2397,
  'token_str': 'homem',
  'sequence': 'Sem medo de ser homem.'}]